In [ ]:
!pip install qiskit[visualization]==1.1.0
!pip install qiskit_ibm_runtime
!pip install matplotlib
!pip install pylatexenc
!pip install networkx

In [3]:
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.synthesis import LieTrotter

from qiskit_ibm_runtime.options import EstimatorOptions, DynamicalDecouplingOptions
from qiskit_ibm_runtime import EstimatorV2, Batch

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from scipy.interpolate import make_interp_spline



In [4]:
# Initial definations

num_spins = 50
#anisotropy = 1.
#h = 1.
dt = Parameter('δt')

In [ ]:
def build_layered_hamiltonian(num_spins, anisotropy, h):

    lattice_map = CouplingMap.from_line(num_spins, bidirectional=False)
    edgelist = lattice_map.graph.edge_list()
    hamlist_even = []
    hamlist_odd = []

    for edge in edgelist:
        if edge[0]%2==0:
            hamlist_even.append(("XX", edge, 1.))
            hamlist_even.append(("YY", edge, 1.))
            hamlist_even.append(("ZZ", edge, anisotropy))
        else:
            hamlist_odd.append(("XX", edge, 1.))
            hamlist_odd.append(("YY", edge, 1.))
            hamlist_odd.append(("ZZ", edge, anisotropy))

    for qubit in lattice_map.physical_qubits:
        hamlist_even.append(("X", [qubit], h))

    hamiltonian_even = SparsePauliOp.from_sparse_list(hamlist_even, num_qubits=num_spins)
    hamiltonian_odd = SparsePauliOp.from_sparse_list(hamlist_odd, num_qubits=num_spins)

    return hamiltonian_even+hamiltonian_odd
print(build_layered_hamiltonian(10, 1, 1))

In [ ]:

service = QiskitRuntimeService("ibm_quantum","API-TOKEN")
backend = service.backend("ibm_sherbrooke")
print(backend)
pass_manager = generate_preset_pass_manager(backend=backend,optimization_level=3)


In [7]:
dt = Parameter('δt')
dt_val = [5*np.pi/2.]
h_vals = np.linspace(0., np.pi/2, 12)
anisotropies = {"Anisotropic":-5., "XXX":1}
num_spins = 50
hamiltonians = {}
time_evolution_operators = {}
trotter_circuits = {}
isa_circuits = {}
isa_z_observables = {}
z_observables = [ SparsePauliOp.from_sparse_list([('Z', [i], 1.)], num_qubits=num_spins) for i in range(num_spins) ]


for phase, anisotropy in anisotropies.items():

    hamiltonians[phase]=[build_layered_hamiltonian(num_spins,anisotropy,h) for h in h_vals]
    time_evolution_operators[phase]=[PauliEvolutionGate(hamiltonians[phase][i], time=dt) for i in range(len(h_vals))]
    trotter_factory = LieTrotter(reps=4)
    trotter_circuits[phase]=[trotter_factory.synthesize(time_evolution_operators[phase][i]) for i in range(len(h_vals))]
    isa_circuits[phase]=[pass_manager.run(trotter_circuits[phase][i]) for i in range(len(h_vals))]
    isa_z_observables[phase]=[[abs_obs.apply_layout(layout=isa_circuits[phase][i].layout) for abs_obs in z_observables] for i in range(len(h_vals))]


In [8]:
all_z_obs_pubs = {}
for phase, circuits in isa_circuits.items():
    all_z_obs_pubs[phase]=[(circuits[i],isa_z_observables[phase][i],dt_val) for i in range(len(circuits))]

In [ ]:
print(circuits[0].depth())
print(circuits[0].count_ops())

In [ ]:
all_z_obs_jobs = {}

with Batch(backend=backend) as batch:
    print(f"Session ID: {batch.session_id}")
    print(f"Backend: {batch.backend()}")

    estimator = EstimatorV2(session=batch)
    for phase,publist in all_z_obs_pubs.items():
        all_z_obs_jobs[phase]=[estimator.run(pubs=[pub]).job_id for pub in publist]

In [ ]:
jobids={}
for phase,jobs in all_z_obs_jobs.items():
    jobids[phase]=[job() for job in jobs]
print(jobids)

In [ ]:
fname = 'idump'
with open(fname, 'w') as file:
    json.dump(jobids, file)

In [ ]:

with open('idump','r') as file:
    joebids=json.load(file)
all_job_data = {}
for phase,jobs in joebids.items():
    all_job_data[phase]=[service.job(job).result() for job in jobs]

print(all_job_data)


In [ ]:
avg_z_data = {}
for phase,results in all_job_data.items():
    avg_z_data[phase]=[np.mean(res[0].data.evs) for res in results]

In [ ]:
sns.set()

h_vals = np.linspace(0., np.pi/2, 12)
x_smooth=np.linspace(h_vals.min(),h_vals.max(),64)
for phase,res in avg_z_data.items():
    spl=make_interp_spline(h_vals,res,k=3)
    smooth_res=spl(x_smooth)
    #optional interpolated graph - not used here
    plt.plot(x_smooth, smooth_res, linestyle='-', label=phase)
plt.xlabel('Magnetic Field Strength(h)')
plt.ylabel('Avg Z')
plt.title('Dynamics of Zi for different Magnetic Field Strength')
plt.legend()
plt.show()

In [ ]:
import datetime
from IPython.display import HTML, display


def qiskit_copyright(line="", cell=None):
    """IBM copyright"""
    now = datetime.datetime.now()

    html = "<div style='width: 100%; background-color:#d5d9e0;"
    html += "padding-left: 10px; padding-bottom: 10px; padding-right: 10px; padding-top: 5px'>"
    html += "<p>&copy; Copyright IBM 2017, %s.</p>" % now.year
    html += "<p>This code is licensed under the Apache License, Version 2.0. You may<br>"
    html += "obtain a copy of this license in the LICENSE.txt file in the root directory<br> "
    html += "of this source tree or at http://www.apache.org/licenses/LICENSE-2.0."

    html += "<p>Any modifications or derivative works of this code must retain this<br>"
    html += "copyright notice, and modified files need to carry a notice indicating<br>"
    html += "that they have been altered from the originals.</p>"
    html += "</div>"
    return display(HTML(html))


qiskit_copyright()